<a href="https://colab.research.google.com/github/maxmatical/fast.ai/blob/master/fastai_%2B_blurr_%2B_deberta_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install --upgrade fastai
!pip install transformers
!pip install ohmeow-blurr

In [2]:
from fastai.test_utils import show_install
show_install()



```text
=== Software === 
python        : 3.7.10
fastai        : 2.2.7
fastcore      : 1.3.19
fastprogress  : 0.2.7
torch         : 1.7.1+cu101
nvidia driver : 460.32
torch cuda    : 10.1 / is available
torch cudnn   : 7603 / is enabled

=== Hardware === 
nvidia gpus   : 1
torch devices : 1
  - gpu0      : Tesla T4

=== Environment === 
platform      : Linux-4.19.112+-x86_64-with-Ubuntu-18.04-bionic
distro        : #1 SMP Thu Jul 23 08:00:38 PDT 2020
conda env     : Unknown
python        : /usr/bin/python3
sys.path      : 
/content
/env/python
/usr/lib/python37.zip
/usr/lib/python3.7
/usr/lib/python3.7/lib-dynload
/usr/local/lib/python3.7/dist-packages
/usr/lib/python3/dist-packages
/usr/local/lib/python3.7/dist-packages/IPython/extensions
/root/.ipython
```

Please make sure to include opening/closing ``` when you paste into forums/github to make the reports appear formatted as code sections.

Optional package(s) to enhance the diagnostics can be installed with:
pip install distro
O

In [3]:
from transformers import *
from fastai.text.all import *

from blurr.data.all import *
from blurr.modeling.all import *

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
path = untar_data(URLs.IMDB_SAMPLE)

model_path = Path('models')
imdb_df = pd.read_csv(path/'texts.csv')

In [5]:
task = HF_TASKS_AUTO.SequenceClassification

pretrained_model_name = 'microsoft/deberta-base'
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR_MODEL_HELPER.get_hf_objects(pretrained_model_name,  task=task)

In [6]:
# single input
blocks = (HF_TextBlock(hf_arch, hf_config, hf_tokenizer, hf_model), CategoryBlock)
dblock = DataBlock(blocks=blocks,  get_x=ColReader('text'), get_y=ColReader('label'), splitter=ColSplitter())

dls = dblock.dataloaders(imdb_df, bs=4)

In [7]:
dls.show_batch(dataloaders=dls, max_n=2)

,text,category
0,"Raising Victor Vargas: A Review<br /><br />You know, Raising Victor Vargas is like sticking your hands into a big, steaming bowl of oatmeal. It's warm and gooey, but you're not sure if it feels right. Try as I might, no matter how warm and gooey Raising Victor Vargas became I was always aware that something didn't quite feel right. Victor Vargas suffers from a certain overconfidence on the director's part. Apparently, the director thought that the ethnic backdrop of a Latino family on the lower east side, and an idyllic storyline would make the film critic proof. He was right, but it didn't fool me. Raising Victor Vargas is the story about a seventeen-year old boy called, you guessed it, Victor Vargas (Victor Rasuk) who lives his teenage years chasing more skirt than the Rolling Stones could do in all the years they've toured. The movie starts off in `Ugly Fat' Donna's bedroom where Victor is sure to seduce her, but a cry from outside disrupts his plans when his best-friend Harold (Kevin Rivera) comes-a-looking for him. Caught in the attempt by Harold and his sister, Victor Vargas runs off for damage control. Yet even with the embarrassing implication that he's been boffing the homeliest girl in the neighborhood, nothing dissuades young Victor from going off on the hunt for more fresh meat. On a hot, New York City day they make way to the local public swimming pool where Victor's eyes catch a glimpse of the lovely young nymph Judy (Judy Marte), who's not just pretty, but a strong and independent too. The relationship that develops between Victor and Judy becomes the focus of the film. The story also focuses on Victor's family that is comprised of his grandmother or abuelita (Altagracia Guzman), his brother Nino (also played by real life brother to Victor, Silvestre Rasuk) and his sister Vicky (Krystal Rodriguez). The action follows Victor between scenes with Judy and scenes with his family. Victor tries to cope with being an oversexed pimp-daddy, his feelings for Judy and his grandmother's conservative Catholic upbringing.<br /><br />The problems that arise from Raising Victor Vargas are a few, but glaring errors. Throughout the film you get to know certain characters like Vicky, Nino, Grandma,",negative
1,"Many neglect that this isn't just a classic due to the fact that it's the first 3D game, or even the first shoot-'em-up. It's also one of the first stealth games, one of the only(and definitely the first) truly claustrophobic games, and just a pretty well-rounded gaming experience in general. With graphics that are terribly dated today, the game thrusts you into the role of B.J.(don't even *think* I'm going to attempt spelling his last name!), an American P.O.W. caught in an underground bunker. You fight and search your way through tunnels in order to achieve different objectives for the six episodes(but, let's face it, most of them are just an excuse to hand you a weapon, surround you with Nazis and send you out to waste one of the Nazi leaders). The graphics are, as I mentioned before, quite dated and very simple. The least detailed of basically any 3D game released by a professional team of creators. If you can get over that, however(and some would suggest that this simplicity only adds to the effect the game has on you), then you've got one heck of a good shooter/sneaking game. The game play consists of searching for keys, health and ammo, blasting enemies(aforementioned Nazis, and a ""boss enemy"" per chapter) of varying difficulty(which, of course, grows as you move further in the game), unlocking doors and looking for secret rooms. There is a bonus count after each level is beaten... it goes by how fast you were(basically, if you beat the 'par time', which is the time it took a tester to go through the same level; this can be quite fun to try and beat, and with how difficult the levels are to find your way in, they are even challenging after many play-throughs), how much Nazi gold(treasure) you coll

In [8]:
model = HF_BaseModelWrapper(hf_model)

learn = Learner(dls, 
                model,
                loss_func=LabelSmoothingCrossEntropyFlat(),
                metrics=[accuracy],
                cbs=[HF_BaseModelCallback],
                splitter=hf_splitter).to_fp16()

learn.unfreeze()

cbs = [
    GradientAccumulation(n_acc=32),
    ReduceLROnPlateau(monitor='accuracy', factor=5, patience=5),
    EarlyStoppingCallback(monitor='accuracy', min_delta=0.01, patience=7),
    SaveModelCallback(monitor="accuracy", fname="model_1")
]



In [9]:
learn.fit_one_cycle(
    3,
    lr_max=3e-5,
    cbs = cbs
)

epoch,train_loss,valid_loss,accuracy,time
0,0.689205,0.690561,0.535000,00:49
1,0.519060,0.368640,0.910000,00:50
2,0.359599,0.343464,0.925000,00:51


Better model found at epoch 0 with accuracy value: 0.5350000262260437.
Better model found at epoch 1 with accuracy value: 0.9100000262260437.
Better model found at epoch 2 with accuracy value: 0.925000011920929.
